In [4]:
import pandas as pd
import seaborn as sb
import pylab as py
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import folium
from folium import plugins
from geopy.geocoders import Nominatim
import datetime
import statsmodels.api as sm

# Purpose: To figure out the increase in cost incurred, for every % increase in Cancer Incidence in Singapore

In [18]:
df = pd.read_csv("health-expenditure.csv")
df

,Year,Operating Expenditure ($m),Development Expenditure ($m),Government Health Expenditure1 ($m),Government Health Expenditure1 (as % of GDP)
0,2006,"1,840",96,2009.0,0.9
1,2007,"2,019",185,2283.2,0.8
2,2008,"2,379",336,2814.1,1.0
3,2009,"2,920",711,3745.8,1.3
4,2010,"3,258",485,3856.7,1.2
5,2011,"3,489",453,4091.5,1.2
6,2012,"4,066",605,4837.3,1.3
7,2013,"5,044",723,5938.1,1.6
8,2014,"5,872","1,147",7223.1,1.8
9,2015,"7,520","1,413",8639.9,2.1


In [28]:
df1 = pd.read_csv("sg-breast-cancer.csv")
df1

,Year,Cancer
0,1972,134.4
1,1977,172.4
2,1982,247.4
3,1987,347.4
4,1992,526.6
5,1997,720.0
6,2002,1116.2
7,2007,1371.4
8,2012,1712.8
9,2017,2167.4


In [29]:
Y = df1["Cancer"]
# Y = df1.astype(float)
X = df1["Year"]

X = sm.add_constant(X)
model = sm.OLS(Y, X)

# Fit the model
model_fit = model.fit()

# # Print the model summary
print(model_fit.summary())

sg_cancer_df = pd.DataFrame()
sg_cancer_df.insert(0, 'Year', range(2006, 2020))
sg_cancer_df["Cancer"] = model_fit.predict(sm.add_constant(sg_cancer_df["Year"]))
sg_cancer_df.to_csv("sg-cancer.csv")

                            OLS Regression Results                            
Dep. Variable:                 Cancer   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     114.5
Date:                Sat, 02 Apr 2022   Prob (F-statistic):           2.03e-06
Time:                        02:43:41   Log-Likelihood:                -74.359
No. Observations:                  11   AIC:                             152.7
Df Residuals:                       9   BIC:                             153.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -9.636e+04   9097.421    -10.592      0.0

C:\Users\colin\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [30]:
df2 = pd.read_csv('sg-cancer.csv')
df2.head()

,Unnamed: 0,Year,Cancer
0,0,2006,1444.423041
1,1,2007,1493.176959
2,2,2008,1541.930876
3,3,2009,1590.684793
4,4,2010,1639.438710


In [31]:
df2 = df2.drop(columns = "Unnamed: 0")

In [35]:
y

,Government Health Expenditure1 ($m)
0,2009
1,2283
2,2814
3,3745
4,3856
5,4091
6,4837
7,5938
8,7223
9,8639


In [33]:
# df2 = df2[df2['Year'] > 1999]
X = df2['Cancer'] * 7000000/100000
X = X.reset_index().drop(columns = 'index')

# df = df[df['Year'] < 2011]
y = df['Government Health Expenditure1 ($m)']
y = y.reset_index().drop(columns = 'index')
y = y.astype(int)

# Define the modell
X = sm.add_constant(X)
model = sm.OLS(y, X)

# Fit the model
model_fit = model.fit()

# Print the model summary
print(model_fit.summary())

                                     OLS Regression Results                                    
Dep. Variable:     Government Health Expenditure1 ($m)   R-squared:                       0.972
Model:                                             OLS   Adj. R-squared:                  0.970
Method:                                  Least Squares   F-statistic:                     416.7
Date:                                 Sat, 02 Apr 2022   Prob (F-statistic):           1.10e-10
Time:                                         02:43:54   Log-Likelihood:                -107.22
No. Observations:                                   14   AIC:                             218.4
Df Residuals:                                       12   BIC:                             219.7
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

C:\Users\colin\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [ ]:
sb.regplot(x=X['%increase'], y=y['Current health expenditure (% of GDP)'])

In [37]:
model_fit.predict([0,1254.0572734956559])/3

array([91.77584077])

# To do: Find % of cancer in SG is Breast Cancer. Then multiply this % to the health expenditure cost. Ideally, also find % of health expenditure cost is on cancer.